In [2]:
!pip install folium
import pandas as pd
import folium
import json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 1.4 MB/s eta 0:00:00 0:00:01


In [4]:
slim_df = pd.read_csv('clean data/airbnb_listings_slim.csv')

# Print the first 5 rows
print(slim_df.head())

# Optional: print basic info
print(slim_df.info())

# Optional: see summary statistics
print(slim_df.describe())

   listing_id        date  price   neighbourhood_cleansed     room_type  \
0      255809  2019-12-05  150.0  De Pijp - Rivierenbuurt  Private room   
1      255809  2019-12-04  150.0  De Pijp - Rivierenbuurt  Private room   
2      255809  2019-12-03  150.0  De Pijp - Rivierenbuurt  Private room   
3      255809  2019-12-02  150.0  De Pijp - Rivierenbuurt  Private room   
4      255809  2019-12-01  150.0  De Pijp - Rivierenbuurt  Private room   

   accommodates  bedrooms  beds  bathrooms  minimum_nights  ...  \
0             4       2.0   4.0        1.0               2  ...   
1             4       2.0   4.0        1.0               2  ...   
2             4       2.0   4.0        1.0               2  ...   
3             4       2.0   4.0        1.0               2  ...   
4             4       2.0   4.0        1.0               2  ...   

   number_of_reviews  review_scores_rating  review_scores_cleanliness  \
0                 60                  97.0                       10.0   


In [ ]:
#Number of rows: 1,200,071
#Columns of interest for mapping:
#neighbourhood_cleansed → for geospatial analysis
#price → could visualize average price per neighborhood
#review_scores_rating, review_scores_cleanliness, reviews_per_month → can analyze quality of listings by area
#Other useful columns: room_type, accommodates, bedrooms, etc., for filtering or aggregating

In [8]:
#loadingGeoJSON file of neighborhoods
import json

# Load GeoJSON file
import pandas as pd
import json
import folium

# Airbnb slim dataset
slim_df = pd.read_csv('clean data/airbnb_listings_slim.csv')

# Neighborhood GeoJSON
with open('raw data/neighbourhoods.geojson', 'r') as f:
    neighborhoods_geo = json.load(f)

In [9]:
#aggregate price by nighbourhood
# Average price per neighborhood
neighborhood_price = slim_df.groupby('neighbourhood_cleansed')['price'].mean().reset_index()
neighborhood_price.head()

,neighbourhood_cleansed,price
0,Bijlmer-Centrum,123.609888
1,Bijlmer-Oost,124.591683
2,Bos en Lommer,138.525612
3,Buitenveldert - Zuidas,232.803146
4,Centrum-Oost,250.256563


In [10]:
# Base map: adjust location and zoom for your city
m = folium.Map(location=[52.3702, 4.8952], zoom_start=12)  # Amsterdam

# Add choropleth
folium.Choropleth(
    geo_data=neighborhoods_geo,
    name='Average Price',
    data=neighborhood_price,
    columns=['neighbourhood_cleansed', 'price'],
    key_on='feature.properties.neighbourhood',  # ensure property matches your GeoJSON
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Price (€)'
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Save map
m.save('amsterdam_avg_price_map.html')

In [ ]:
#Geographic Analysis of Airbnb Listings

#Map shows how average prices vary across neighborhoods.

#Central/popular areas have higher prices; some neighborhoods have few listings.

#Location strongly influences price and availability.

In [11]:
# Example: average price per neighborhood
neighborhood_summary = slim_df.groupby('neighbourhood_cleansed')['price'].mean().reset_index()
neighborhood_summary.rename(columns={'price': 'avg_price'}, inplace=True)

# Save as CSV
neighborhood_summary.to_csv('neighborhood_avg_price.csv', index=False)